### Historical Stock Return Data and Portfolio Construction
#### This code is an example to download stock price data from Yahoo finance and to use the historical returns to construct an optimal portfolio by using mean-variance optimization. 
#### The optimization method uses the average stock returns, and the covariances of historical returns to solve for the portfolio mix that achieves the best value of (Expected Portfolio Return / Volatility of Portfolio Return) ratio.  

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from cvxopt import matrix, solvers

# Step 1: Collect stock prices and the S&P 500 Index
tickers = ["C", "V", "NFLX", "KO", "XOM"] #['IAU', 'VDE', 'XLB', 'DBC', 'CQQQ', '^GSPC']
data = yf.download(tickers, start='2018-12-01', end='2021-12-31', interval='1d')['Adj Close']

# Step 2: Calculate monthly returns
monthly_prices = data.resample('M').last()
monthly_returns = monthly_prices.pct_change().dropna()

# Separate S&P 500 returns from stock returns
sp500_returns = monthly_returns['^GSPC']
stock_returns = monthly_returns.drop(columns=['^GSPC'])

# Step 3: Estimate stock betas using regression
betas = {}
for stock in stock_returns.columns:
    X = sp500_returns.values.reshape(-1, 1)
    y = stock_returns[stock].values
    reg = LinearRegression().fit(X, y)
    betas[stock] = reg.coef_[0]

# Sort stocks by beta and select the 5 highest beta stocks
beta_sorted = sorted(betas.items(), key=lambda x: x[1], reverse=True)
top_5_stocks = [stock for stock, beta in beta_sorted[:5]]

# Plot the beta values as a bar chart
plt.figure(figsize=(10, 6))
plt.bar(betas.keys(), betas.values(), color='blue')
plt.title('Estimated Betas for Each Stock')
plt.xlabel('Stocks')
plt.ylabel('Beta Value')
plt.show()

# Step 4: Mean-Variance Portfolio Optimization for top 5 beta stocks
top_5_returns = stock_returns[top_5_stocks]

# Calculate expected returns (mean returns) and covariance matrix
mu = top_5_returns.mean().values
cov_matrix = top_5_returns.cov().values

# Define the Mean-Variance Optimization problem using cvxopt
n = len(mu)
P = matrix(cov_matrix)  # Covariance matrix
q = matrix(np.zeros(n))  # No linear term (for minimizing variance)

# Weight constraints: 5% <= weights <= 35%
G = matrix(np.vstack((-np.eye(n), np.eye(n))))  # Inequality matrix
h = matrix(np.hstack(([-0.05] * n, [0.35] * n)))  # Lower and upper bounds for the weights

# Equality constraint: sum of weights = 1
A = matrix(1.0, (1, n))  # Equality constraint matrix
b = matrix(1.0)  # Bound for the equality constraint

# Solve the optimization problem
sol = solvers.qp(P, q, G, h, A, b)
weights = np.array(sol['x']).flatten()

# Step 5: Display the optimized portfolio weights
optimized_weights = dict(zip(top_5_stocks, weights))
print("Optimized Portfolio Weights:")
for stock, weight in optimized_weights.items():
    print(f"{stock}: {weight:.2%}")

# Step 6: Plot the pie chart of stock weights
plt.figure(figsize=(8, 6))
plt.pie(optimized_weights.values(), labels=optimized_weights.keys(), autopct='%1.1f%%', startangle=140)
plt.title('Optimized Portfolio Stock Weights')
plt.show()


[*********************100%***********************]  5 of 5 completed


KeyError: '^GSPC'